In [ ]:
from ase.build import nanotube
from ase.visualize import view
from ase.io import write
import numpy as np

In [4]:
# Parameters
n, m = 6, 0
ncells = 30
bond = 1.42


In [24]:
def create_nanotube(n=6, m=0, n_unit_cells=1, bond=1.42, vacuum=8.0):
    """Create a CNT with PBC in x, vacuum in y and z, and proper cell."""
    atoms = nanotube(n=n, m=m, length=n_unit_cells, bond=bond, verbose=False)

    # Rotate tube axis to align with x
    atoms.rotate('z', 'x', rotate_cell=False, center='COP')

    # Set PBC only in x-direction
    atoms.set_pbc((True, False, False))

    # Manually determine the x-extent of the tube
    positions = atoms.get_positions()
    x_min = np.min(positions[:, 0])
    x_max = np.max(positions[:, 0])
    length_x = x_max - x_min
    print(f"Length in x: {length_x:.2f} Å")
    # Define the simulation cell: periodic in x, vacuum in y/z
    cell = np.array([
        [length_x, 0, 0],
        [0, 2 * vacuum, 0],
        [0, 0, 2 * vacuum]
    ])
    atoms.set_cell(cell, scale_atoms=False)

    # Center atoms in y and z (not x, to preserve periodicity)
    atoms.center(axis=(0, 1, 2))

    return atoms


In [25]:
cnt = create_nanotube(n=n, m=m, n_unit_cells=ncells, bond=bond, vacuum=20.0)
view(cnt, viewer="x3d", show_unit_cell=True, show_bonds=True, bond_radius=0.1)


Length in x: 127.09 Å


In [26]:
# create xyz file with CNT structure
write("cnt.xyz", cnt, format="vasp")